## Övning 5
__John Landeholt__

johnlan@kth.se


__agenda:__

* Kattis (Lab 8)
    * Testning
* Syntax (Lab 9)
* Automater
* Reguljära uttryck (regex)

### Kattis

<img src="img/ö5i1.png" width="33%" style="float:right"/>

Kattis är en plattform som kontrollerar ditt programs `korrekthet`. Den gör det igenom att köra ditt program på olika testfall, och om den får ett resultat den förväntar sig, så är programmet korrekt utfört, annars inte.


I __labb 9__ ska vi använda oss utav [Formelkoll](https://kth.kattis.com/problems/kth.tilda.formelkoll2) som kommer att läsa en molekylformel med hjälp av ditt program och sedan jämföra det med sin databas för att se om det stämmer.

>__notera__: Kattis är verkligen inte perfekt, pedagogiskt eller lätt. Hon kommer endast berätta att det är fel och inte varför det är fel. Det är upp dit datalogen att förstå vad som kan vara fel.

Hjälpkod för att läsa och skriva till Kattis:

[Kattio](https://github.com/landeholt/tilda/blob/main/help-code/kattio.py)

[Skelettkod till lab 8 och 9](https://github.com/landeholt/tilda/blob/main/skeleton-code/molecule.py)

In [ ]:
# Kattio
from sys import stdin, stdout

def get_formula():
    return stdin.readline().strip()

def get_formulas():
    for line in stdin:
        yield line.strip()

def send(out):
    stdout.write(str(out) + '\n')

### Testning

Innan man skickar in till __kattis__ kan det vara bra och faktiskt testa sin kod lokalt först. Detta gör man genom att skriva testfiler som kontrollerar om inputen resulterar till den förväntade outputen.

[testfil till lab 8 och 9](https://github.com/landeholt/tilda/blob/main/help-code/test_lab8.py)

In [ ]:
from YOUR_FILE import YOUR_FUNCTION as f
# from lab8 import formula as f
import pytest

@pytest.mark.parametrize('formula, expected', [
    ('Na', 'Formeln är syntaktiskt korrekt'),
    ('H2O', 'Formeln är syntaktiskt korrekt'),
    ('Si(C3(COOH)2)4(H2O)7', 'Formeln är syntaktiskt korrekt'),
    ('Na332', 'Formeln är syntaktiskt korrekt'),
    ('C(Xx4)5', 'Okänd atom vid radslutet 4)5'),
    ('C(OH4)C', 'Saknad siffra vid radslutet C'),
    ('C(OH4C', 'Saknad högerparentes vid radslutet'),
    ('H2O)Fe', 'Felaktig gruppstart vid radslutet )Fe'),
    ('H0', 'För litet tal vid radslutet'),
    ('H1C', 'För litet tal vid radslutet C'),
    ('H02C', 'För litet tal vid radslutet 2C'),
    ('Nacl', 'Saknad stor bokstav vid radslutet cl'),
    ('a', 'Saknad stor bokstav vid radslutet a'),
    ('(Cl)2)3', 'Felaktig gruppstart vid radslutet )3'),
    (')', 'Felaktig gruppstart vid radslutet )'),
    ('2', 'Felaktig gruppstart vid radslutet 2'),
    ('', 'Formeln är syntaktiskt korrekt'),
    ('H111', 'Formeln är syntaktiskt korrekt'),
    ('H110', 'Formeln är syntaktiskt korrekt')
])

def test_formulas(formula, expected):
    cert = f(formula)
    assert cert == expected

In [2]:
!pytest test_lab8.py

============================= test session starts ==============================
platform darwin -- Python 3.9.0, pytest-6.2.1, py-1.10.0, pluggy-0.13.1
rootdir: /Users/John/Documents/KTH/projects/stora-boken-om-nada
collected 19 items                                                             

test_lab8.py ...................                                         [100%]

============================== 19 passed in 0.04s ==============================


In [ ]:
!pytest tast

### Syntax

__vad är syntax?__

Det är en uppsättning regler för vilken `ordning` något får ske. När `python` läser in ditt program och analyserar det innan körningen så sker det en syntaktiskt kontroll för att säkertställa att programmet är `giltigt`. Men bara för att något är giltigt, betyder det inte att det är `korrekt` eller för den delen `ändligt`.

En syntax kan vara rekursiv.

    <mol> ::= <group> | <group><mol>
    <group> ::= <atom> |<atom><num> | (<mol>) <num>
    ...
    
Vilket i det här fallet innebär att en molekyl kan vara __en__ eller __flera__ grupper.